In [3]:
import torch
dataset_name = 'en'
task_embedding = torch.load(f'outputs/{dataset_name}/task_embedding.pt')
data_embedding = torch.load(f'outputs/{dataset_name}/all_data_embedding.pt')

In [50]:
import pandas as pd
import os
task_list = pd.read_csv(f'../data/{dataset_name}/task.csv', sep='\t')['task'].tolist()
job_posting = pd.read_csv(f'../data/{dataset_name}/all_data.csv', sep='\t')

In [1]:
import torch

def cosine_similarity_matrix(x, y):
    x_norm = x / x.norm(dim=1, keepdim=True)
    y_norm = y / y.norm(dim=1, keepdim=True)
    return torch.mm(x_norm, y_norm.t())

x = torch.rand(5, 128)  
y = torch.rand(6, 128)
similarity_matrix = cosine_similarity_matrix(x, y)
print(similarity_matrix)

tensor([[0.7178, 0.6964, 0.7900, 0.7459, 0.7067, 0.7808],
        [0.7102, 0.7350, 0.7449, 0.7424, 0.6959, 0.7526],
        [0.7723, 0.7618, 0.7679, 0.7894, 0.7644, 0.7775],
        [0.7163, 0.7372, 0.7317, 0.7480, 0.7657, 0.7605],
        [0.7501, 0.7029, 0.8097, 0.7442, 0.7219, 0.8001]])


In [58]:
similarity_matrix = cosine_similarity_matrix(data_embedding, task_embedding)
topk_indices = torch.topk(similarity_matrix, dim=-1,k=10).indices
topk_value = torch.topk(similarity_matrix, dim=-1,k=10).values

In [60]:
similarity_matrix

tensor([[0.9231, 0.9488, 0.9293,  ..., 0.9210, 0.9215, 0.9477],
        [0.9057, 0.9305, 0.9074,  ..., 0.8815, 0.8799, 0.9364],
        [0.9171, 0.9284, 0.9170,  ..., 0.9432, 0.9542, 0.9325],
        ...,
        [0.6588, 0.6659, 0.6286,  ..., 0.6161, 0.6836, 0.7257],
        [0.9009, 0.9310, 0.9042,  ..., 0.8692, 0.8563, 0.9216],
        [0.8200, 0.8393, 0.8042,  ..., 0.7645, 0.7845, 0.8636]])

In [59]:
import pandas as pd
topk_task = pd.DataFrame(topk_indices.numpy())
topk_values = pd.DataFrame(topk_value.numpy())

In [46]:
for col in topk_task.columns:
    job_posting[f"task{col}"] = topk_task[col].apply(lambda x: task_list[x]) 
    job_posting[f"task{col}_sim"] = topk_values[col] 
job_posting.to_csv(f'outputs/{dataset_name}/topk_task.csv', index=None, sep='\t')
job_posting

,occupation,text,task0,task1,task2,task3,task4,task5,task6,task7,...,task0_sim,task1_sim,task2_sim,task3_sim,task4_sim,task5_sim,task6_sim,task7_sim,task8_sim,task9_sim
0,Real Estate,Job descriptionA leading real estate firm in N...,Establish client goals or objectives for music...,Drive vehicles with capacities under three ton...,"Obtain specimens, cultivating, isolating, and ...","Arrange for applicable duties, taxes, or paper...","Manage subcontractor activities, reviewing pro...",Translate nursing practice information between...,Participate in design and production of multim...,"Operate engine lathe to grind, file, and turn ...",...,0.965208,0.962782,0.962408,0.961720,0.961521,0.960858,0.960442,0.960163,0.959572,0.959541
1,Wellness and Fitness Services,"At Aspen Therapy and Wellness , we are committ...","Supervise students' fieldwork, internship, and...",Participate in design and production of multim...,"Supervise and train other technologists, techn...",Drive vehicles with capacities under three ton...,"Investigate alleged parole violations, using i...",Monitor stored biofuels products or secondary ...,"Start pumps to wash and rinse reactor vessels,...","Supervise and train other drafters, technologi...",...,0.956688,0.955799,0.955396,0.952893,0.951952,0.950531,0.950175,0.949921,0.949823,0.949190
2,Restaurants,The National Exemplar is accepting application...,"Add cutlery, napkins, food, and other items to...","Locate library materials for patrons, includin...","Stock bar with beer, wine, liquor, and related...","Gather, register, and arrange the material to ...",Assign cases for students to hear and try,"Select flora and foliage for arrangements, wor...",Monitor and log transmitter readings,"Fill orders for products and samples, followin...",...,0.976446,0.974522,0.973335,0.973004,0.970808,0.970159,0.968801,0.968350,0.968346,0.968043
3,Law Practice,Senior Associate Attorney - Elder Law / Trusts...,"Obtain, cut, stain, and mount biological mater...","Consult with medical, business, and community ...","Attend meetings, conventions, and civic events...","Instruct patients and families in work, social...","Confer with clients to determine objectives, b...",Direct public health education programs dealin...,Process print and non-print library materials ...,"Collaborate with specialists, such as rehabili...",...,0.985203,0.984977,0.983990,0.983895,0.983874,0.983805,0.983746,0.983673,0.983628,0.983465
4,Wellness and Fitness Services,Looking for HVAC service tech with experience ...,"Design layout of craft interior, including car...",Take class attendance and maintain attendance ...,Take class attendance and maintain attendance ...,Take class attendance and maintain attendance ...,"Select the appropriate drill for the job, usin...",Determine traffic areas and decide location of...,Select appropriate culturing system or procedu...,Determine and implement the best layout to ach...,...,0.992375,0.991899,0.991899,0.991899,0.991897,0.991691,0.991634,0.991549,0.991500,0.991495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107492,Law Practice,Our Walnut Creek office is currently seeking a...,Direct clerical staff to keep records of expor...,Prepare biological specimens such as amniotic ...,Interview prospective clients to obtain data a...,"Direct the training of firefighters, assigning...","Supervise the work of technologists, technicia...",Monitor stored biofuels products or secondary ...,"Supervise law students, volunteers, or other p...",Supervise biological technologists and technic...,...,0.976661,0.973474,0.972898,0.972440,0.972303,0.971844,0.971798,0.971621,0.971416,0.971219
107493,Software Development,About Pinterest:Millions of people across the ...,Drive vehicles with capacities under three ton...,"Supervise and train other drafters, technologi...","Start machinery, such as pumps, feeders, or co...","Analyze data from biofuels studies, such as fl...","Investigate alleged parole violati

In [49]:
df = job_posting[job_posting['task9_sim']>0.99]
df.to_excel(f'outputs/{dataset_name}/sample_99.xlsx')
df

,occupation,text,task0,task1,task2,task3,task4,task5,task6,task7,...,task0_sim,task1_sim,task2_sim,task3_sim,task4_sim,task5_sim,task6_sim,task7_sim,task8_sim,task9_sim
4,Wellness and Fitness Services,Looking for HVAC service tech with experience ...,"Design layout of craft interior, including car...",Take class attendance and maintain attendance ...,Take class attendance and maintain attendance ...,Take class attendance and maintain attendance ...,"Select the appropriate drill for the job, usin...",Determine traffic areas and decide location of...,Select appropriate culturing system or procedu...,Determine and implement the best layout to ach...,...,0.992375,0.991899,0.991899,0.991899,0.991897,0.991691,0.991634,0.991549,0.991500,0.991495
10,Food and Beverage Services,Glastender Inc. is a family-owned manufacturer...,"Establish clear objectives for all lessons, un...","Establish clear objectives for all lessons, un...","Establish clear objectives for all lessons, un...","Establish clear objectives for all lessons, un...","Establish clear objectives for all lessons, un...","Establish clear objectives for all lessons, un...","Establish clear objectives for all lessons, un...","Establish clear objectives for all lessons, un...",...,0.994029,0.994029,0.994029,0.994029,0.994029,0.993900,0.993900,0.993623,0.993623,0.993623
23,Wellness and Fitness Services,Sheet metal fabricator for in house shopWeldin...,Greet and seat customers,Lead tours and answer visitors' questions abou...,Clean and polish removable appliances,Set up and operate portable field transmission...,Schedule parties and take reservations,Take class attendance and maintain attendance ...,Take class attendance and maintain attendance ...,Take class attendance and maintain attendance ...,...,0.993935,0.993708,0.993346,0.993216,0.993058,0.992986,0.992986,0.992986,0.992851,0.992805
97,Construction,Primary duties include:Developing projects bud...,"Prepare cost and labor estimates, based on cal...",Coordinate equipment operation with presentati...,"Direct team activities, establishing task prio...",Manage intradepartmental infection control and...,"Compound, package, and label pharmaceutical pr...","Plan, supervise, and review work of assigned s...",Direct activities of workers conducting autops...,"Clean engines, sediment bulk and screens, and ...",...,0.990879,0.990819,0.990593,0.990583,0.990507,0.990471,0.990427,0.990308,0.990275,0.990234
405,IT Services and IT Consulting,Position SummaryThis is a full-time on-site ro...,Determine the type of geothermal loop system m...,Schedule and supervise the construction and te...,"Determine reference points, machine cutting pa...",Supervise laboratory sessions and field work a...,Interpret the results of all methods of non-de...,Define and compare possible radio frequency id...,Examine blast areas to determine amounts and k...,"Lay out, position, and apply shock-absorbing, ...",...,0.993939,0.993108,0.993020,0.992812,0.992798,0.992645,0.992611,0.992570,0.992546,0.992429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105908,IT Services and IT Consulting,"Location: Aiken, SCDUTIESSupport the Nuclear &...","Prepare reports on biomass plant operations, s...",Determine the nature of code violations and ac...,Supervise and direct the work of correctional ...,Monitor control panels during pumping operatio...,Provide to supervisors oral and written report...,"Verify that heating, ventilating, and air cond...",Develop rehabilitation programs for assigned o...,"Assign duties to inmates, providing instructio...",...,0.994294,0.994221,0.994138,0.993663,0.993407,0.993072,0.993067,0.993067,0.992887,0.992683
106087,Computer and Network Security,"Job DescriptionKLDiscovery, one of the largest...",Determine the nature of code violations and ac...,"Take, receive, or check periodic inmate counts",Direct the operation and organization of kitch...,"Establish clear objectives for all lessons, un...","Establish cl